## Alcholic Anonymous News articles extraction from The Tribune. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, urls = [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'Alcoholics Anonymous site:www.tribuneindia.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
max_pages = round([int(s) for s in soup.select_one('#resultStats').text.split() if s.isdigit()][0]/10)

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

176 <class 'list'>


### Iterates urls through for loop. Scraping the Articles using BeautifulSoup4 which extracts above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')

        try:
            # Extracts the Headlines 
            headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
        except:
            headlines.append(None)
            
        try:
            # Extracts the Descriptions
            descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
        except:
            descriptions.append(None)
            
        try:
            # Extracts the Authors
            authors.append(soup.select_one('meta[name="author"]')['content'])
        except:
            authors.append(None)
            
        try:
            # Extracts the published dates
            dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
        except:
            dates.append(None)

        try:
            # Extracts the news articles
            news.append(soup.select_one('.storyText').text.replace('\n', ''))
        except:
            news.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 2min 24sribuneindia.com/2014/20140408/dun.htmpmanship/276411.htmlverdict-on-panamagate-today/394557.html406505.htmlml/619408.html


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(urls))

176 176 176 176 176 176


### Creating a csv file after checking array length and if to url error creates NaN values in the rows and dropping it

In [10]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news})
tbl = tbl.dropna()
tbl.to_csv('The_Tribune.csv', index=False)
tbl.head()

,Headlines,Descriptions,Authors,Published_Dates,Articles
0,Hope for alcoholics through AA: Experts,CHANDIGARH: There is a hope for hopeless alcoh...,Tribune News Service,2016-08-09T02:36:00,"Chandigarh, August 8There is a hope for hopele..."
1,When she hits the bottle...,"Today, globally India shows the third largest ...",Tribune News Service,2016-07-31T00:58:00,"Linda Chhakchhuak\rToday, globally India shows..."
3,Raising a toast to life,How about a life beyond Patiala peg! For those...,Tribune News Service,2017-05-04T02:27:00,Pradeep SharmaHow about a life beyond Patiala ...
4,"Poor me, don’t pour that drink...","AAP MP Bhagwant Mann put alcohol right, left, ...",,2019-02-04T06:37:00,Lakhinder Jit Singh Vohra\rAAP MP Bhagwant Man...
5,Adopt ‘alcohol-free’ lifestyle for healthy lif...,CHANDIGARH:Experts and practitioners from spec...,Tribune News Service,2017-07-31T01:34:00,"Chandigarh, July 30\rExperts and practitioners..."
